In [1]:
# STEP 1 - GATHERING DATA
import pandas as pd
import numpy as np

seasons_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MSeasons.csv')
#print(seasons_df.head())

conferences_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/Conferences.csv')
#print(conferences_df.head())

team_conferences_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MTeamConferences.csv')
#print(team_conferences_df.head())

tourney_seeds_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneySeeds.csv')
#print(tourney_seeds_df.head())

tourney_slots_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneySlots.csv')
#print(tourney_slots_df.head())

tourney_seed_round_slots_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneySeedRoundSlots.csv')
#print(tourney_seed_round_slots_df.head())

regseason_compact_results_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')
#print(regseason_compact_results_df.head())

tourney_compact_results_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
#print(tourney_compact_results_df.head())

regseason_detailed_results_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
#print(regseason_detailed_results_df.head())

tourney_detailed_results_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')
#print(tourney_detailed_results_df.head())

conference_tourney_games_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MConferenceTourneyGames.csv')
#print(conference_tourney_games_df.head())

massey_ordinals_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MMasseyOrdinals.csv')
#print(massey_ordinals_df.head())

sample_submission_df = pd.read_csv('ncaam-march-mania-2021/MDataFiles_Stage1/MSampleSubmissionStage1.csv')
#print(sample_submission_df.head())

In [2]:

#REGULAR SEASON FEATURES


In [3]:
#Part 1 - Obtain TeamID, Season, Win %, Games Played, and PPG

#Find the game duration 
regseason_compact_results_df['GameDuration'] = regseason_compact_results_df['NumOT']
for i in range(len(regseason_compact_results_df.NumOT.value_counts())):
    regseason_compact_results_df.loc[regseason_compact_results_df['NumOT'] == regseason_compact_results_df.NumOT.value_counts().index[i], 'GameDuration'] = 40 + i*5

regseason_compact_results_df['Wppg'] = regseason_compact_results_df['WScore'] * (40.0 / regseason_compact_results_df['GameDuration'])
regseason_compact_results_df['Lppg'] = regseason_compact_results_df['LScore'] * (40.0 / regseason_compact_results_df['GameDuration'])

regseason_compact_results_df = regseason_compact_results_df.drop(['DayNum', 'WLoc', 'NumOT', 'WScore','LScore','GameDuration'],axis = 1)
#print(regseason_compact_results_df.head()) 

   Season  WTeamID  LTeamID  Wppg  Lppg
0    1985     1228     1328  81.0  64.0
1    1985     1106     1354  77.0  70.0
2    1985     1112     1223  63.0  56.0
3    1985     1165     1432  70.0  54.0
4    1985     1192     1447  86.0  74.0


In [4]:
#Calculate average points per game and number of games for winning and losing team
Wppg_df = regseason_compact_results_df[["Season","WTeamID",'Wppg']].groupby(["Season","WTeamID"]).agg(['mean', 'count'])
Lppg_df = regseason_compact_results_df[["Season","LTeamID",'Lppg']].groupby(["Season","LTeamID"]).agg(['mean', 'count'])
Wppg_df.columns = Wppg_df.columns.droplevel(0)
Wppg_df = Wppg_df.reset_index().rename(columns = {'mean':'Wppg', 'count':'WGames'})
Lppg_df.columns = Lppg_df.columns.droplevel(0)
Lppg_df = Lppg_df.reset_index().rename(columns = {'mean':'Lppg', 'count':'LGames'})
#print(Wppg_df)
#print(Lppg_df)

In [5]:
#Merge PPG for wins and losses and fill NaNs with 0s for undefeated and no-win teams
ppg_df = pd.merge(Wppg_df, Lppg_df, left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'], how='outer')
ppg_df = ppg_df.fillna({'WTeamID': ppg_df.LTeamID, 'Wppg': 0, 'WGames': 0, 'LTeamID': ppg_df.WTeamID , 'Lppg': 0, 'LGames': 0})
#print(ppg_df)

In [6]:
#Calculate PPG and win percentage for season 
ppg_df['PPG'] = (ppg_df['Wppg']*ppg_df['WGames'] + ppg_df['Lppg']*ppg_df['LGames'])/(ppg_df['WGames'] + ppg_df['LGames'])
ppg_df['WPerc'] = (ppg_df['WGames'])/(ppg_df['WGames'] + ppg_df['LGames'])
ppg_df['TeamID'] = ppg_df['WTeamID'].astype('int32')
ppg_df['GamesPlayed'] = (ppg_df['WGames'] + ppg_df['LGames'])
ppg_df = ppg_df.drop(['WTeamID', 'Wppg' ,'WGames','LTeamID', 'Lppg','LGames'],axis = 1)
#print(ppg_df.head())

In [7]:
#Part 2 - Obtain game level statistics:
    #2-PT field goal efficiency (2-PT field goals made per game / 2-PT field goals attempted per game)
    #3-PT field goal efficiency
    #Total field goal efficiency
    #Free throw efficiency 
    #Assists per game
    #Blocks per game
    #Steals per game
    #Turnovers per game
    #Personal fouls per game
    #Offensive rebounds per game
    #Defensive rebounds per game
    #Points per shot
    #Total rebounds per game
    #Average turnovers to assists ratio
    #Average turnovers to steals ratio
    #Average scoring margin


#Gametime normalization 
regseason_detailed_results_df['GameDuration'] = regseason_detailed_results_df['NumOT']
for i in range(len(regseason_detailed_results_df.NumOT.value_counts())):
    regseason_detailed_results_df.loc[regseason_detailed_results_df['NumOT'] == regseason_detailed_results_df.NumOT.value_counts().index[i], 'GameDuration'] = 40 + i*5

regseason_detailed_results_df['WFGM'] = (regseason_detailed_results_df['WFGM'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WFGA'] = (regseason_detailed_results_df['WFGA'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WFGM3'] = (regseason_detailed_results_df['WFGM3'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WFGA3'] = (regseason_detailed_results_df['WFGA3'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WFTM'] = (regseason_detailed_results_df['WFTM'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WFTA'] = (regseason_detailed_results_df['WFTA'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WOR'] = (regseason_detailed_results_df['WOR'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WDR'] = (regseason_detailed_results_df['WDR'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WAst'] = (regseason_detailed_results_df['WAst'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WTO'] = (regseason_detailed_results_df['WTO'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WStl'] = (regseason_detailed_results_df['WStl'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WBlk'] = (regseason_detailed_results_df['WBlk'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['WPF'] = (regseason_detailed_results_df['WPF'] / regseason_detailed_results_df['GameDuration']) * 40

regseason_detailed_results_df['LFGM'] = (regseason_detailed_results_df['LFGM'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LFGA'] = (regseason_detailed_results_df['LFGA'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LFGM3'] = (regseason_detailed_results_df['LFGM3'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LFGA3'] = (regseason_detailed_results_df['LFGA3'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LFTM'] = (regseason_detailed_results_df['LFTM'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LFTA'] = (regseason_detailed_results_df['LFTA'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LOR'] = (regseason_detailed_results_df['LOR'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LDR'] = (regseason_detailed_results_df['LDR'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LAst'] = (regseason_detailed_results_df['LAst'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LTO'] = (regseason_detailed_results_df['LTO'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LStl'] = (regseason_detailed_results_df['LStl'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LBlk'] = (regseason_detailed_results_df['LBlk'] / regseason_detailed_results_df['GameDuration']) * 40
regseason_detailed_results_df['LPF'] = (regseason_detailed_results_df['LPF'] / regseason_detailed_results_df['GameDuration']) * 40

#Calculate game level statistics for winning team 

regseason_detailed_results_df["W2FG%"] = (regseason_detailed_results_df["WFGM"] / regseason_detailed_results_df["WFGA"])
regseason_detailed_results_df["W3FG%"] = (regseason_detailed_results_df["WFGM3"] / regseason_detailed_results_df["WFGA3"])
regseason_detailed_results_df["WFG%"] = (regseason_detailed_results_df["WFGM"] + regseason_detailed_results_df["WFGM3"] ) / (regseason_detailed_results_df["WFGA"] + regseason_detailed_results_df["WFGA3"])
regseason_detailed_results_df["WFT%"] = (regseason_detailed_results_df['WFTM'] / regseason_detailed_results_df['WFTA']) 
regseason_detailed_results_df["WScoreMargin"] = regseason_detailed_results_df["WScore"] - regseason_detailed_results_df["LScore"]
regseason_detailed_results_df["WPPS"] = (regseason_detailed_results_df["WScore"]) / (regseason_detailed_results_df["WFGA"] + regseason_detailed_results_df["WFGA3"])
regseason_detailed_results_df["W#POS"] = regseason_detailed_results_df["WFGA"] + regseason_detailed_results_df["WFGA3"] - regseason_detailed_results_df['WOR'] + regseason_detailed_results_df['WTO'] + (0.4 * regseason_detailed_results_df['WFTA'])
regseason_detailed_results_df["WOEFF"] = (regseason_detailed_results_df["WScore"]) / (regseason_detailed_results_df["W#POS"])
regseason_detailed_results_df["WDEFF"] = (regseason_detailed_results_df["LScore"]) / (regseason_detailed_results_df["W#POS"])

#Calculate game level statistics for losing team 

regseason_detailed_results_df["L2FG%"] = (regseason_detailed_results_df["LFGM"] / regseason_detailed_results_df["LFGA"])
regseason_detailed_results_df["L3FG%"] = (regseason_detailed_results_df["LFGM3"] / regseason_detailed_results_df["LFGA3"])
regseason_detailed_results_df["LFG%"] = (regseason_detailed_results_df["LFGM"] + regseason_detailed_results_df["LFGM3"] ) / (regseason_detailed_results_df["LFGA"] + regseason_detailed_results_df["LFGA3"])
regseason_detailed_results_df["LFT%"] = (regseason_detailed_results_df['LFTM'] / regseason_detailed_results_df['LFTA']) 
regseason_detailed_results_df["LScoreMargin"] = (regseason_detailed_results_df["LScore"] - regseason_detailed_results_df["WScore"])
regseason_detailed_results_df["LPPS"] = (regseason_detailed_results_df["LScore"]) / (regseason_detailed_results_df["LFGA"] + regseason_detailed_results_df["LFGA3"])
regseason_detailed_results_df["L#POS"] = regseason_detailed_results_df["LFGA"] + regseason_detailed_results_df["LFGA3"] - regseason_detailed_results_df['LOR'] + regseason_detailed_results_df['LTO'] + (0.4 * regseason_detailed_results_df['LFTA'])
regseason_detailed_results_df["LOEFF"] = (regseason_detailed_results_df["LScore"]) / (regseason_detailed_results_df["L#POS"])
regseason_detailed_results_df["LDEFF"] = (regseason_detailed_results_df["WScore"]) / (regseason_detailed_results_df["L#POS"])
#print(regseason_detailed_results_df.head())
#for col in regseason_detailed_results_df.columns:
    #print(col)
    
    

In [8]:
#Extract relevant columns and rename 
Wstats_df = regseason_detailed_results_df[['Season', 
                                           'DayNum', 
                                           'WTeamID', 
                                           'WScore', 
                                           'WFGM', 
                                           'WFGA', 
                                           'WFGM3', 
                                           'WFGA3', 
                                           'WFTM', 
                                           'WFTA', 
                                           'WOR', 
                                           'WDR', 
                                           'WAst', 
                                           'WTO', 
                                           'WStl', 
                                           'WBlk', 
                                           'WPF', 
                                           'W2FG%', 
                                           'W3FG%', 
                                           'WFG%', 
                                           'WFT%', 
                                           'WScoreMargin',
                                           'WPPS',
                                           'WOEFF',
                                           'WDEFF']]

Wstats_df = Wstats_df.rename(columns = {'WTeamID': 'TeamID',
                                        'WScore': 'AvgScore', 
                                           'WFGM': 'FGM', 
                                           'WFGA': 'FGA', 
                                           'WFGM3': 'FGM3', 
                                           'WFGA3': 'FGA3', 
                                           'WFTM': 'FTM', 
                                           'WFTA': 'FTA', 
                                           'WOR': 'OR', 
                                           'WDR': 'DR', 
                                           'WAst': 'Ast', 
                                           'WTO': 'TO', 
                                           'WStl': 'Stl', 
                                           'WBlk': 'Blk', 
                                           'WPF': 'PF', 
                                           'W2FG%': '2FG%', 
                                           'W3FG%': '3FG%', 
                                           'WFG%': 'FG%', 
                                           'WFT%': 'FT%', 
                                           'WScoreMargin': 'AvgScoreMarg',
                                           'WPPS': 'PPS',
                                           'WOEFF': 'OEFF',
                                           'WDEFF': 'DEFF'})

Lstats_df = regseason_detailed_results_df[['Season', 
                                           'DayNum', 
                                           'LTeamID', 
                                           'LScore', 
                                           'LFGM', 
                                           'LFGA', 
                                           'LFGM3', 
                                           'LFGA3', 
                                           'LFTM', 
                                           'LFTA', 
                                           'LOR', 
                                           'LDR', 
                                           'LAst', 
                                           'LTO', 
                                           'LStl', 
                                           'LBlk', 
                                           'LPF', 
                                           'L2FG%', 
                                           'L3FG%', 
                                           'LFG%', 
                                           'LFT%', 
                                           'LScoreMargin',
                                           'LPPS',
                                           'LOEFF',
                                           'LDEFF']]

Lstats_df = Lstats_df.rename(columns = {'LTeamID': 'TeamID', 
                                           'LScore': 'AvgScore', 
                                           'LFGM': 'FGM', 
                                           'LFGA': 'FGA', 
                                           'LFGM3': 'FGM3', 
                                           'LFGA3': 'FGA3', 
                                           'LFTM': 'FTM', 
                                           'LFTA': 'FTA', 
                                           'LOR': 'OR', 
                                           'LDR': 'DR', 
                                           'LAst': 'Ast', 
                                           'LTO': 'TO', 
                                           'LStl': 'Stl', 
                                           'LBlk': 'Blk', 
                                           'LPF': 'PF', 
                                           'L2FG%': '2FG%', 
                                           'L3FG%': '3FG%', 
                                           'LFG%': 'FG%', 
                                           'LFT%': 'FT%', 
                                           'LScoreMargin': 'AvgScoreMarg',
                                           'LPPS': 'PPS',
                                           'LOEFF': 'OEFF',
                                           'LDEFF': 'DEFF'})
stats_df = pd.concat([Wstats_df, Lstats_df]) 
#print(stats_df.head())


In [9]:
#Take seasonal and 30 day averages of the efficiency metrics
seasonAvgStats_df = stats_df.groupby(by = ['Season', 'TeamID']).agg('mean').reset_index().drop(['DayNum'], axis = 1)
#print(seasonAvgStats_df.head())

In [10]:
#Part 4 - Add Ranking Data
rankings_df = massey_ordinals_df[massey_ordinals_df['RankingDayNum'] == 133]
rankings_df = rankings_df.reset_index().drop(['index','RankingDayNum'], axis = 1)
mean_df = rankings_df.groupby(by = ['Season','TeamID'])[['OrdinalRank']].mean().reset_index()
mean_df = mean_df.rename(columns = {'OrdinalRank':'MeanRank'})
massey_ordinals_df = rankings_df[rankings_df['SystemName'] == 'MAS']
massey_ordinals_df = massey_ordinals_df.reset_index().drop(['index','SystemName'], axis = 1)
massey_ordinals_df = massey_ordinals_df.rename(columns = {'OrdinalRank':'MasseyRank'})
pomeroy_df = rankings_df[rankings_df['SystemName'] == 'POM']
pomeroy_df = pomeroy_df.reset_index().drop(['index','SystemName'], axis = 1)
pomeroy_df = pomeroy_df.rename(columns = {'OrdinalRank':'PomeroyRank'})
rankings_df = pd.merge(massey_ordinals_df, pomeroy_df, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], how='left')
rankings_df = pd.merge(rankings_df, mean_df, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], how='left')
#print(rankings_df.head())

In [11]:
#Part 5 - Merge ppg_df, seasonAvgStats_df, and rankings_df
features_df = pd.merge(ppg_df, seasonAvgStats_df)
features_df = pd.merge(features_df, rankings_df)
print(features_df.head())
for col in features_df.columns:
    print(col)

   Season        PPG     WPerc  TeamID  GamesPlayed   AvgScore        FGM  \
0    2003  57.250000  0.428571    1102         28.0  57.250000  19.142857   
1    2003  75.967901  0.481481    1103         27.0  78.777778  26.268313   
2    2003  69.015873  0.607143    1104         28.0  69.285714  23.940476   
3    2003  70.400855  0.269231    1105         26.0  71.769231  23.938462   
4    2003  63.281746  0.464286    1106         28.0  63.607143  23.309524   

         FGA      FGM3       FGA3  ...      3FG%       FG%       FT%  \
0  39.785714  7.821429  20.821429  ...  0.367637  0.449465  0.642402   
1  54.051029  5.296296  15.611523  ...  0.331990  0.455683  0.735271   
2  56.964286  6.329365  19.805556  ...  0.325442  0.395264  0.705168   
3  60.519658  7.447009  20.373504  ...  0.359630  0.386880  0.709598   
4  55.019841  6.083333  17.567460  ...  0.350196  0.408169  0.623158   

   AvgScoreMarg       PPS      OEFF      DEFF  MasseyRank  PomeroyRank  \
0      0.250000  0.969331  0.7

In [12]:
#TOURNAMENT FEATURES

In [13]:
#CHOOSE MODELS



In [14]:
#TRAIN MODEL



In [15]:
#EVALUATE THE MODEL



In [16]:
#STEP 6 - TUNE THE PARAMETERS



In [17]:
#STEP 7 - CREATE FINAL PREDICTIONS FOR 2021 TOURNAMENT 

